# Building an image classification model using very little data  

This tutorial presents several ways to build an image classifier using Keras from just a few hundred or thousand pictures from each class you want to be able to recognize.

We will go over the following options:  

- training a small network from scratch (as a baseline)  
- using the bottleneck features of a pre-trained network  
- fine-tuning the top layers of a pre-trained network  
  
This will lead us to cover the following Keras features:   
  
- fit_generator for training Keras a model using Python data generators  
- ImageDataGenerator for real-time data augmentation  
- layer freezing and model fine-tuning  
- ...and more.  


## Step.1 - Install libs

In [ ]:
!conda install --name root tensorflow -y

In [ ]:
from distutils.version import StrictVersion
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

# should at least be 1.0
assert StrictVersion(tf.__version__) >= StrictVersion('1.0.0')

tf.__version__

In [ ]:
!pip install keras

In [ ]:
import keras as keras

# should at least be 1.2 and use TensorFlow backend
assert StrictVersion(keras.__version__) >= StrictVersion('1.2.2')

keras.__version__

In [ ]:
!python -m pip install -U numpy
!python -m pip install -U scipy
!python -m pip install -U Pillow

## Data

Data is a subset of the dogs vs cats dataset
https://www.kaggle.com/c/dogs-vs-cats/data  
Made a folder stucture and added the relevant data.

### Folder structure

```python
data/
    train/
        dogs/ ### 1024 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 1024 pictures
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/ ### 416 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 416 pictures
            cat001.jpg
            cat002.jpg
            ...
```

The github repo includes about 3000 images, but the holde dataset is about 12500.

### Data loading

In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

### Imports

In [3]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 2048 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


# What inspired Convolutional Networks?

CNNs are biologically-inspired models inspired by research by D. H. Hubel and T. N. Wiesel. They proposed an explanation for the way in which mammals visually perceive the world around them using a layered architecture of neurons in the brain, and this in turn inspired engineers to attempt to develop similar pattern recognition mechanisms in computer vision.

![alt text](https://qph.ec.quoracdn.net/main-qimg-235acb60a481423eaf70c39b17bc914b.webp "Logo Title Text 1")

In their hypothesis, within the visual cortex, complex functional responses generated by "complex cells" are constructed from more simplistic responses from "simple cells'. 

For instances, simple cells would respond to oriented edges etc, while complex cells will also respond to oriented edges but with a degree of spatial invariance.

Receptive fields exist for cells, where a cell responds to a summation of inputs from other local cells.

The architecture of deep convolutional neural networks was inspired by the ideas mentioned above 
- local connections 
- layering  
- spatial invariance (shifting the input signal results in an equally shifted output signal. , most of us are able to recognize specific faces under a variety of conditions because we learn abstraction These abstractions are thus invariant to size, contrast, rotation, orientation
 
However, it remains to be seen if these computational mechanisms of convolutional neural networks are similar to the computation mechanisms occurring in the primate visual system

- convolution operation
- shared weights
- pooling/subsampling 

## How does it work? 

![alt text](https://images.nature.com/w926/nature-assets/srep/2016/160610/srep27755/images_hires/srep27755-f1.jpg "Logo Title Text 1")
![alt text](https://www.mathworks.com/content/mathworks/www/en/discovery/convolutional-neural-network/jcr:content/mainParsys/image_copy.adapt.full.high.jpg/1497876372993.jpg "Logo Title Text 1")

### Step 2 - Convolution 

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/more_images/Convolution_schematic.gif "Logo Title Text 1")

![alt text](http://xrds.acm.org/blog/wp-content/uploads/2016/06/Figure_2.png "Logo Title Text 1")

- A convolution is an orderly procedure where two sources of information are intertwined.

- A kernel (also called a filter) is a smaller-sized matrix in comparison to the input dimensions of the image, that consists of real valued entries.

- Kernels are then convolved with the input volume to obtain so-called ‘activation maps’ (also called feature maps).  
- Activation maps indicate ‘activated’ regions, i.e. regions where features specific to the kernel have been detected in the input. 

- The real values of the kernel matrix change with each learning iteration over the training set, indicating that the network is learning to identify which regions are of significance for extracting features from the data.

- We compute the dot product between the kernel and the input matrix. -The convolved value obtained by summing the resultant terms from the dot product forms a single entry in the activation matrix. 

- The patch selection is then slided (towards the right, or downwards when the boundary of the matrix is reached) by a certain amount called the ‘stride’ value, and the process is repeated till the entire input image has been processed. - The process is carried out for all colour channels.

- instead of connecting each neuron to all possible pixels, we specify a 2 dimensional region called the ‘receptive field[14]’ (say of size 5×5 units) extending to the entire depth of the input (5x5x3 for a 3 colour channel input), within which the encompassed pixels are fully connected to the neural network’s input layer. It’s over these small regions that the network layer cross-sections (each consisting of several neurons (called ‘depth columns’)) operate and produce the activation map. (reduces computational complexity)

![alt text](http://i.imgur.com/g4hRI6Z.png "Logo Title Text 1")
![alt text](http://i.imgur.com/tpQvMps.jpg "Logo Title Text 1")
![alt text](http://i.imgur.com/oyXkhHi.jpg "Logo Title Text 1")
![alt text](http://xrds.acm.org/blog/wp-content/uploads/2016/06/Figure_5.png "Logo Title Text 1")

###  Step 3 - Pooling
![alt text](http://xrds.acm.org/blog/wp-content/uploads/2016/06/Figure_6.png "Logo Title Text 1")

- Pooling reducing the spatial dimensions (Width x Height) of the Input Volume for the next Convolutional Layer. It does not affect the depth dimension of the Volume.  
- The transformation is either performed by taking the maximum value from the values observable in the window (called ‘max pooling’), or by taking the average of the values. Max pooling has been favoured over others due to its better performance characteristics.
- also called downsampling

###  Step 4 - Normalization (ReLU in our case)

![alt text](http://xrds.acm.org/blog/wp-content/uploads/2016/06/CodeCogsEqn-3.png "Logo Title Text 1")

Normalization (keep the math from breaking by turning all negative numbers to 0)  (RELU) a stack of images becomes a stack of images with no negative values. 

Repeat Steps 2-4 several times. More, smaller images (feature maps created at every layer)

### Step 5 - Regularization 

- Dropout forces an artificial neural network to learn multiple independent representations of the same data by alternately randomly disabling neurons in the learning phase.
- Dropout is a vital feature in almost every state-of-the-art neural network implementation.
- To perform dropout on a layer, you randomly set some of the layer's values to 0 during forward propagation.

See [this](http://iamtrask.github.io/2015/07/28/dropout/)

![alt text](https://i.stack.imgur.com/CewjH.png "Logo Title Text 1")

###  Step 6 - Probability Conversion

At the very end of our network (the tail), we'll apply a softmax function to convert the outputs to probability values for each class. 

![alt text](https://1.bp.blogspot.com/-FHDU505euic/Vs1iJjXHG0I/AAAAAAABVKg/x4g0FHuz7_A/s1600/softmax.JPG "Logo Title Text 1")

## So how do we learn the magic numbers? 

- We can learn features and weight values through backpropagation

![alt text](http://www.robots.ox.ac.uk/~vgg/practicals/cnn/images/cover.png "Logo Title Text 1")

![alt text](https://image.slidesharecdn.com/cnn-toupload-final-151117124948-lva1-app6892/95/convolutional-neural-networks-cnn-52-638.jpg?cb=1455889178 "Logo Title Text 1")

The other hyperparameters are set by humans and they are an active field of research (finding the optimal ones)

i.e -  number of neurons, number of features, size of features, poooling window size, window stride

## Small Conv Net

### Model architecture definition

In [4]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,...)`
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # Remove the CWD from sys.path while we load stuff.


We are here using a Sequential model since to make the network 

In [5]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Training

In [6]:
nb_epoch = 30
nb_train_samples = 2048
nb_validation_samples = 832

In [7]:
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=128, epochs=30, validation_steps=832)`
  


Epoch 1/30
128/128 [==============================] - 248s - loss: 0.7023 - acc: 0.5376 - val_loss: 0.6534 - val_acc: 0.6370
Epoch 2/30
128/128 [==============================] - 251s - loss: 0.6410 - acc: 0.6523 - val_loss: 0.6467 - val_acc: 0.6346
Epoch 3/30
128/128 [==============================] - 252s - loss: 0.5958 - acc: 0.6914 - val_loss: 0.5991 - val_acc: 0.6970
Epoch 4/30
128/128 [==============================] - 249s - loss: 0.5379 - acc: 0.7271 - val_loss: 0.6053 - val_acc: 0.6909
Epoch 5/30
128/128 [==============================] - 250s - loss: 0.4901 - acc: 0.7676 - val_loss: 0.5597 - val_acc: 0.7381
Epoch 6/30
128/128 [==============================] - 249s - loss: 0.4339 - acc: 0.7959 - val_loss: 0.6061 - val_acc: 0.7203
Epoch 7/30
128/128 [==============================] - 259s - loss: 0.3799 - acc: 0.8306 - val_loss: 0.5857 - val_acc: 0.7205
Epoch 8/30
128/128 [==============================] - 256s - loss: 0.3265 - acc: 0.8623 - val_loss: 0.6181 - val_acc: 0.7358


In [10]:
model.save_weights('models/basic_cnn_20_epochs.h5')

In [11]:
#model.load_weights('models_trained/basic_cnn_20_epochs.h5')

If your model successfully runs at one epoch, go back and it for 30 epochs by changing nb_epoch above.  I was able to get to an val_acc of 0.71 at 30 epochs.
A copy of a pretrained network is available in the pretrained folder.

### Evaluating on validation set

Computing loss and accuracy :

In [12]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[2.0198164684518884, 0.70905949519230771]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_no_dataaugmentation.png)

**After ~10 epochs the neural network reach ~70% accuracy. We can witness overfitting, no progress is made over validation set in the next epochs**

## Data augmentation for improving the model

By applying random transformation to our train set, we artificially enhance our dataset with new unseen images.  
This will hopefully reduce overfitting and allows better generalization capability for our network.

Example of data augmentation applied to a picture:
![Example of data augmentation applied to a picture](pictures/cat_data_augmentation.png)

In [13]:
train_datagen_augmented = ImageDataGenerator(
        rescale=1./255,        # normalize pixel values to [0,1]
        shear_range=0.2,       # randomly applies shearing transformation
        zoom_range=0.2,        # randomly applies shearing transformation
        horizontal_flip=True)  # randomly flip the images

# same code as before
train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 2048 images belonging to 2 classes.


In [ ]:
model.fit_generator(
        train_generator_augmented,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=64, epochs=30, validation_steps=832)`
  


Epoch 1/30
64/64 [==============================] - 250s - loss: 0.6496 - acc: 0.6899 - val_loss: 0.5904 - val_acc: 0.6994
Epoch 2/30
64/64 [==============================] - 253s - loss: 0.6266 - acc: 0.6870 - val_loss: 0.5643 - val_acc: 0.7514
Epoch 3/30
64/64 [==============================] - 249s - loss: 0.5731 - acc: 0.7412 - val_loss: 0.7995 - val_acc: 0.5404
Epoch 4/30
64/64 [==============================] - 258s - loss: 0.5494 - acc: 0.7285 - val_loss: 0.5845 - val_acc: 0.7220
Epoch 5/30
64/64 [==============================] - 278s - loss: 0.5466 - acc: 0.7432 - val_loss: 0.6365 - val_acc: 0.7562
Epoch 6/30
64/64 [==============================] - 289s - loss: 0.5717 - acc: 0.7339 - val_loss: 0.5228 - val_acc: 0.7428
Epoch 7/30
64/64 [==============================] - 273s - loss: 0.5261 - acc: 0.7588 - val_loss: 0.6112 - val_acc: 0.7569
Epoch 8/30
64/64 [==============================] - 266s - loss: 0.5234 - acc: 0.7695 - val_loss: 0.5437 - val_acc: 0.7554
Epoch 9/30
64/64

In [8]:
model.save_weights('models/augmented_30_epochs.h5')

In [15]:
#model.load_weights('models_trained/augmented_30_epochs.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [16]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.69096329235113585, 0.76802884615384615]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 100 epochs :

![Accuracy evolution](pictures/scores_with_dataaugmentation.png)

**Thanks to data-augmentation, the accuracy on the validation set improved to ~80%**

## Using a pre-trained model

The process of training a convolutionnal neural network can be very time-consuming and require a lot of datas.  

We can go beyond the previous models in terms of performance and efficiency by using a general-purpose, pre-trained image classifier.  This example uses VGG16, a model trained on the ImageNet dataset - which contains millions of images classified in 1000 categories. 

On top of it, we add a small multi-layer perceptron and we train it on our dataset.

### VGG16 + small MLP
![VGG16 + Dense layers Schema](pictures/vgg16_original.png)

#### VGG16 model architecture definition

In [17]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

#### Loading VGG16 weights
This part is a bit complicated because the structure of our model is not exactly the same as the one used when training weights.  
Otherwise, we would use the `model.load_weights()` method.

In [18]:
import h5py
f = h5py.File('models/vgg16_weights.h5')
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    layer.set_weights(weights)

f.close()

### Using the VGG16 model to process samples

In [19]:
train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

Found 2048 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


This is a long process, so we save the output of the VGG16 once and for all.  

In [20]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, nb_train_samples)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [21]:
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, nb_validation_samples)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

Now we can load it...

In [22]:
train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

And define and train the custom fully connected neural network :

In [23]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
nb_epoch=40
model_top.fit(train_data, train_labels,
          nb_epoch=nb_epoch, batch_size=32,
          validation_data=(validation_data, validation_labels))

Train on 2048 samples, validate on 832 samples
Epoch 1/40
2048/2048 [==============================] - 0s - loss: 0.9784 - acc: 0.6855 - val_loss: 0.4140 - val_acc: 0.8077
Epoch 2/40
2048/2048 [==============================] - 0s - loss: 0.4831 - acc: 0.7891 - val_loss: 0.3646 - val_acc: 0.8365
Epoch 3/40
2048/2048 [==============================] - 0s - loss: 0.3974 - acc: 0.8271 - val_loss: 0.3369 - val_acc: 0.8558
Epoch 4/40
2048/2048 [==============================] - 0s - loss: 0.3438 - acc: 0.8525 - val_loss: 0.3136 - val_acc: 0.8594
Epoch 5/40
2048/2048 [==============================] - 0s - loss: 0.3131 - acc: 0.8647 - val_loss: 0.5777 - val_acc: 0.7873
Epoch 6/40
2048/2048 [==============================] - 0s - loss: 0.2883 - acc: 0.8823 - val_loss: 0.2913 - val_acc: 0.8786
Epoch 7/40
2048/2048 [==============================] - 0s - loss: 0.2321 - acc: 0.9048 - val_loss: 0.3724 - val_acc: 0.8462
Epoch 8/40
2048/2048 [==============================] - 0s - loss: 0.2057 - ac

The training process of this small neural network is very fast : ~2s per epoch

In [25]:
model_top.save_weights('models/bottleneck_40_epochs.h5')

### Bottleneck model evaluation

In [26]:
#model_top.load_weights('models/with-bottleneck/1000-samples--100-epochs.h5')
#model_top.load_weights('/notebook/Data1/Code/keras-workshop/models/with-bottleneck/1000-samples--100-epochs.h5')

Loss and accuracy :

In [27]:
model_top.evaluate(validation_data, validation_labels)

704/832 [========================>.....] - ETA: 0s

[0.782802758165277, 0.87980769230769229]

Evolution of accuracy on training (blue) and validation (green) sets for 1 to 32 epochs :

![Accuracy evolution](pictures/scores_with_bottleneck.png)

**We reached a 90% accuracy on the validation after ~1m of training (~20 epochs) and 8% of the samples originally available on the Kaggle competition !**

In [28]:
##Fine-tuning the top layers of a a pre-trained network

Start by instantiating the VGG base and loading its weights.

In [30]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

In [31]:
import h5py
f = h5py.File('models/vgg/vgg16_weights.h5')
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    layer.set_weights(weights)

f.close()

Build a classifier model to put on top of the convolutional model. For the fine tuning, we start with a fully trained-classifer. We will use the weights from the earlier model. And then we will add this model on top of the convolutional base.

In [32]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model_vgg.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.load_weights('models/bottleneck_40_epochs.h5')

model_vgg.add(top_model)

For fine turning, we only want to train a few layers.  This line will set the first 25 layers (up to the conv block) to non-trainable.

In [33]:
for layer in model_vgg.layers[:25]:
    layer.trainable = False

In [34]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model_vgg.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [35]:
# prepare data augmentation configuration  . . . do we need this?
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

Found 2048 images belonging to 2 classes.
Found 832 images belonging to 2 classes.


In [36]:
# fine-tune the model
model_vgg.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

Epoch 1/40
2048/2048 [==============================] - 39s - loss: 0.3593 - acc: 0.8862 - val_loss: 0.3669 - val_acc: 0.8738
Epoch 2/40
2048/2048 [==============================] - 35s - loss: 0.2559 - acc: 0.9077 - val_loss: 0.3483 - val_acc: 0.8594
Epoch 3/40
2048/2048 [==============================] - 35s - loss: 0.1942 - acc: 0.9292 - val_loss: 0.3380 - val_acc: 0.8618
Epoch 4/40
2048/2048 [==============================] - 35s - loss: 0.1900 - acc: 0.9258 - val_loss: 0.3268 - val_acc: 0.8858
Epoch 5/40
2048/2048 [==============================] - 35s - loss: 0.1638 - acc: 0.9395 - val_loss: 0.3094 - val_acc: 0.8894
Epoch 6/40
2048/2048 [==============================] - 35s - loss: 0.1307 - acc: 0.9502 - val_loss: 0.3038 - val_acc: 0.8930
Epoch 7/40
2048/2048 [==============================] - 35s - loss: 0.1223 - acc: 0.9556 - val_loss: 0.3267 - val_acc: 0.8990
Epoch 8/40
2048/2048 [==============================] - 35s - loss: 0.1233 - acc: 0.9570 - val_loss: 0.2864 - val_acc:

In [37]:
model_vgg.save_weights('models/finetuning_20epochs_vgg.h5')

In [38]:
model_vgg.load_weights('models/finetuning_20epochs_vgg.h5')

### Evaluating on validation set

Computing loss and accuracy :

In [39]:
model_vgg.evaluate_generator(validation_generator, nb_validation_samples)

[0.46926221093879295, 0.90384615384615385]

In [40]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.69096328031558252, 0.76802884615384615]

In [41]:
model_top.evaluate(validation_data, validation_labels)

704/832 [========================>.....] - ETA: 0s

[0.782802758165277, 0.87980769230769229]